In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch

In [2]:
base_path_train = "inaturalist_12K/train" 
input_shape = (224,224)
id2label = {}
label2id = {}
label_list = []
X = []
y = []
for id,label in enumerate(os.listdir(base_path_train)):
    if label[0]==".":
        continue
    id2label[id] = label
    label2id[label] = id
    label_list.append(label)
    label_path = os.path.join(base_path_train,label)
    for img_path in os.listdir(label_path):
        if img_path == ".DS_Store":
            continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X.append(img)
        y.append(id)
    print(f"id:{id}, Label: {label} done")
X = np.array(X).transpose(0, 3, 1, 2)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
print(X.shape)
print(y.shape)

id:1, Label: Amphibia done
id:2, Label: Animalia done
id:3, Label: Arachnida done
id:4, Label: Aves done
id:5, Label: Fungi done
id:6, Label: Insecta done
id:7, Label: Mammalia done
id:8, Label: Mollusca done
id:9, Label: Plantae done
id:10, Label: Reptilia done
torch.Size([9999, 3, 224, 224])
torch.Size([9999])


In [3]:
base_path_test = "inaturalist_12K/val" 
input_shape = (224,224)
X_test = []
y_test = []
for id,label in enumerate(os.listdir(base_path_test)):
    if label[0]==".":
        continue
    label_path = os.path.join(base_path_test,label)
    for img_path in os.listdir(label_path):
        if img_path == ".DS_Store":
            continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X_test.append(img)
        y_test.append(id+1)
    print(f"Test data id:{id+1}, Label: {label} done")
X_test = np.array(X_test).transpose(0, 3, 1, 2)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
print(X_test.shape)
print(y_test.shape)

Test data id:1, Label: Amphibia done
Test data id:2, Label: Animalia done
Test data id:3, Label: Arachnida done
Test data id:4, Label: Aves done
Test data id:5, Label: Fungi done
Test data id:6, Label: Insecta done
Test data id:7, Label: Mammalia done
Test data id:8, Label: Mollusca done
Test data id:9, Label: Plantae done
Test data id:10, Label: Reptilia done
torch.Size([2000, 3, 224, 224])
torch.Size([2000])


In [4]:
print(id2label)
print(label2id)
print(label_list)

{1: 'Amphibia', 2: 'Animalia', 3: 'Arachnida', 4: 'Aves', 5: 'Fungi', 6: 'Insecta', 7: 'Mammalia', 8: 'Mollusca', 9: 'Plantae', 10: 'Reptilia'}
{'Amphibia': 1, 'Animalia': 2, 'Arachnida': 3, 'Aves': 4, 'Fungi': 5, 'Insecta': 6, 'Mammalia': 7, 'Mollusca': 8, 'Plantae': 9, 'Reptilia': 10}
['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


## Part-A

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms

In [6]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
train_data = [data for data in zip(X_train,y_train)]
val_data = [data for data in zip(X_val,y_val)]
test_data = [data for data in zip(X_test,y_test)]
# train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [7]:
class customCNN(nn.Module):
    def __init__(self,m=6,k=3,n=256,activation_func="ReLU"):
        super().__init__()
        self.k = k # size of filters (kxk)
        self.n = n # no.of neurons in the dense layer
        self.m = m # no. of filters
        self.conv1 = nn.Conv2d(3,self.m,self.k)
        self.conv = nn.Conv2d(self.m,self.m,self.k)
        self.maxpool = nn.MaxPool2d(2,2)
        self.fc1 = nn.LazyLinear(self.n)
        self.fc2 = nn.Linear(self.n, 10)
        activations = {
            "mish": nn.Mish(),
            "gelu": nn.GELU(),
            "silu": nn.SiLU(),
            "relu": nn.ReLU()
        }
        self.activation = activations.get(activation_func.lower(), nn.ReLU())
    def forward(self,x,n_conv=5):
        for i in range(n_conv):
            if i==0:
                x = self.conv1(x)
            x = self.conv(x)
            x = self.activation(x)
            x = self.maxpool(x)
        x = torch.flatten(x,1)
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        output = F.softmax(x,dim=1)
        return output

In [8]:
model = customCNN()
x = torch.randn(99, 3, 224, 224)
out = model(x)  # should output [9999, 10]
print(out.shape)  # torch.Size([9999, 10])

torch.Size([99, 10])


In [83]:
import torch.optim as optim
model = customCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
outputs = model(X_train[0])

In [77]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        print(inputs)
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

tensor([[[[187, 133, 109],
          [184, 130, 106],
          [184, 130, 106],
          ...,
          [232, 187, 168],
          [234, 188, 172],
          [235, 189, 174]],

         [[233, 192, 174],
          [232, 191, 173],
          [232, 189, 172],
          ...,
          [163, 132,  86],
          [164, 130,  85],
          [173, 135,  90]],

         [[188, 144,  95],
          [182, 140,  92],
          [172, 131,  85],
          ...,
          [ 59,  48,  16],
          [ 61,  51,  16],
          [ 62,  52,  17]],

         ...,

         [[ 15,  13,  14],
          [ 15,  13,  14],
          [ 14,  12,  13],
          ...,
          [231, 215, 215],
          [231, 215, 215],
          [231, 215, 215]],

         [[231, 215, 215],
          [231, 215, 215],
          [231, 215, 215],
          ...,
          [129, 162,  75],
          [126, 160,  73],
          [123, 159,  69]],

         [[127, 161,  85],
          [134, 166,  90],
          [134, 165,  85],
         

RuntimeError: Input type (unsigned char) and bias type (float) should be the same